# Динамика доли каналов в течение дня
Пример расчета доли в разбивке по часам

## Описание задачи и условий расчета
- Период: 01.09.2022 - 31.10.2022 по месяцам
- Временной интервал: 05:00-29:00 с разбивкой по часам
- ЦА: 4+ и 25-49
- Место просмотра: Все места (дом+дача)
- Каналы: Рен ТВ, ТНТ									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

In [ ]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["РЕН ТВ", "ТНТ"])

# далее в запросах будем использовать id телесетей.

In [ ]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-09-01', '2022-10-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (60, 83)'

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName','researchMonth', 'timeBand60']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [ ]:
# Задаем ЦА по которым будет рассчитан отчет
targets = {
    '4+':'age >= 4',
    '25-49':'age >= 25 AND age <= 49'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{target}" 
    basedemo_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

Настраиваем внешний вид таблицы

In [ ]:
df = df.astype(dtype={'timeBand60':int}) #меняем формат для корректной сортировки

df_res = pd.pivot_table(df, values=['Share'],
                        index=['timeBand60'], 
                        columns=['tvNetName','prj_name','researchMonth'])
df_res

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('05_share_during_the_day')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)